In [1]:
!unzip phi2-mentalhealth.zip

Archive:  phi2-mentalhealth.zip
   creating: phi2-mentalhealth/
  inflating: phi2-mentalhealth/tokenizer.json  
  inflating: phi2-mentalhealth/merges.txt  
  inflating: phi2-mentalhealth/adapter_config.json  
  inflating: phi2-mentalhealth/special_tokens_map.json  
  inflating: phi2-mentalhealth/vocab.json  
  inflating: phi2-mentalhealth/README.md  
  inflating: phi2-mentalhealth/tokenizer_config.json  
  inflating: phi2-mentalhealth/added_tokens.json  
  inflating: phi2-mentalhealth/adapter_model.safetensors  
   creating: phi2-mentalhealth/checkpoint-100/
  inflating: phi2-mentalhealth/checkpoint-100/training_args.bin  
  inflating: phi2-mentalhealth/checkpoint-100/tokenizer.json  
  inflating: phi2-mentalhealth/checkpoint-100/merges.txt  
  inflating: phi2-mentalhealth/checkpoint-100/adapter_config.json  
  inflating: phi2-mentalhealth/checkpoint-100/special_tokens_map.json  
  inflating: phi2-mentalhealth/checkpoint-100/vocab.json  
  inflating: phi2-mentalhealth/checkpoint-100/RE

In [2]:
!ls phi2-mentalhealth

adapter_config.json	   checkpoint-150	    tokenizer_config.json
adapter_model.safetensors  merges.txt		    tokenizer.json
added_tokens.json	   README.md		    vocab.json
checkpoint-100		   special_tokens_map.json


In [3]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ Load fine-tuned model
model_path = "phi2-mentalhealth"  # change path if needed

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
model.eval()

# ✅ Define chat function
def chat(message, history):
    # Format prompt from chat history
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"<|user|>\n{user_msg}\n<|assistant|>\n{bot_msg}\n"
    prompt += f"<|user|>\n{message}\n<|assistant|>\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the assistant's last message
    return response.split("<|assistant|>")[-1].strip()

# ✅ Create Gradio interface
gr.ChatInterface(
    fn=chat,
    title="🧠 YouMatter - Mental Health Chatbot",
    description="Talk to your fine-tuned Phi-2 chatbot 💬",
    theme="soft"
).launch(share=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bdf918a4ec68f66ca3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
